In [3]:
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import gesture_recognizer

import matplotlib.pyplot as plt


# https://developers.google.com/mediapipe/solutions/customization/gesture_recognizer

# insert training and test data import here

# dataset path
dataset_path = os.getcwd() + "/Training Data/Gesture Image Data/" # insert local path here
print(dataset_path)

/home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/


In [4]:
# dataset labels
labels = []
for i in os.listdir(dataset_path):
  if os.path.isdir(os.path.join(dataset_path, i)):
    labels.append(i)
print(labels)

['H', 'I', 'none']


In [5]:
# splits dataset
data = gesture_recognizer.Dataset.from_folder(
    dirname=dataset_path,
    hparams=gesture_recognizer.HandDataPreprocessingParams(
        shuffle=True,
        min_detection_confidence=0.8)
)
train_data, rest_data = data.split(0.8)
validation_data, test_data = rest_data.split(0.5)

Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/I/I2909.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/I/I1149.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/H/H32.jpg


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/I/901.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/I/I294.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/H/H2874.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/I/105.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/I/I2773.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/I/I1707.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/H/H1999.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/H/H1514.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/I/I2994.jpg
INFO:tensorflow:Loading image /home/daksh/AutoSlug/asl/Training Data/Gesture Image Data/H/H887.jpg
INFO:t

2023-09-12 19:31:54.222008: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-12 19:31:54.222593: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder
INFO:tensorflow:Load valid hands with size: 7265, num_label: 3, labels: none,H,I.


INFO:tensorflow:Load valid hands with size: 7265, num_label: 3, labels: none,H,I.


In [7]:
# train model
hparams = gesture_recognizer.HParams(
    export_dir="exported_model",                              
    epochs=100,
    batch_size=2
)
options = gesture_recognizer.GestureRecognizerOptions(hparams=hparams)
model = gesture_recognizer.GestureRecognizer.create(
    train_data=train_data,
    validation_data=validation_data,
    options=options
)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hand_embedding (InputLayer  [(None, 128)]             0         
 )                                                               
                                                                 
 batch_normalization_1 (Bat  (None, 128)               512       
 chNormalization)                                                
                                                                 
 re_lu_1 (ReLU)              (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 custom_gesture_recognizer_  (None, 3)                 387       
 out (Dense)                                                     
                                                           

INFO:tensorflow:Training the models...


Resuming from exported_model/epoch_models/model-0056
Epoch 1/100
2906/2906 [==============================] - 7s 2ms/step - loss: 0.1590 - categorical_accuracy: 0.7694 - val_loss: 0.1215 - val_categorical_accuracy: 0.8347 - lr: 0.0010
Epoch 2/100
2906/2906 [==============================] - 6s 2ms/step - loss: 0.1425 - categorical_accuracy: 0.7920 - val_loss: 0.1453 - val_categorical_accuracy: 0.7355 - lr: 9.9000e-04
Epoch 3/100
2906/2906 [==============================] - 6s 2ms/step - loss: 0.1372 - categorical_accuracy: 0.7916 - val_loss: 0.2206 - val_categorical_accuracy: 0.6515 - lr: 9.8010e-04
Epoch 4/100
2906/2906 [==============================] - 6s 2ms/step - loss: 0.1359 - categorical_accuracy: 0.7949 - val_loss: 0.2240 - val_categorical_accuracy: 0.6336 - lr: 9.7030e-04
Epoch 5/100
2906/2906 [==============================] - 6s 2ms/step - loss: 0.1358 - categorical_accuracy: 0.7954 - val_loss: 0.3679 - val_categorical_accuracy: 0.5248 - lr: 9.6060e-04
Epoch 6/100
2906/2906

In [9]:
# evaluates performance/loss
loss, acc = model.evaluate(test_data, batch_size=1)
print(f"Test loss:{loss}, Test accuracy:{acc}")

727/727 [==============================] - 2s 1ms/step - loss: 8.3079 - categorical_accuracy: 0.4649
Test loss:8.30793571472168, Test accuracy:0.46492433547973633


In [10]:
# export trained model
model.export_model() # insert steps here idk what, but it requires the model to be exported as a TFlite file

Using existing files at /tmp/model_maker/gesture_recognizer/gesture_embedder.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/palm_detection_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/hand_landmark_full.tflite
Using existing files at /tmp/model_maker/gesture_recognizer/canned_gesture_classifier.tflite
INFO:tensorflow:Assets written to: /tmp/tmp_a199ihn/saved_model/assets


INFO:tensorflow:Assets written to: /tmp/tmp_a199ihn/saved_model/assets
2023-09-12 19:47:05.646192: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-09-12 19:47:05.646218: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-09-12 19:47:05.646552: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp_a199ihn/saved_model
2023-09-12 19:47:05.647269: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2023-09-12 19:47:05.647279: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp_a199ihn/saved_model
2023-09-12 19:47:05.648615: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2023-09-12 19:47:05.649213: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-09-12 19:47:05.666553: I tensorflow/cc/saved_model/